In [93]:
import os
import numpy as np
import h5py
import glob
import itertools
import sys
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.autograd.variable import *
import torch.optim as optim

import random
import generatorIN
from generatorIN import InEventLoader
import time
args_cuda = bool(sys.argv[2])


In [16]:
# define the pytorch model
class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden, De, Do,
                 fr_activation=0, fo_activation=0, fc_activation=0, optimizer = 0, verbose = False):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = De
        self.Dx = 0
        self.Do = Do
        self.n_targets = n_targets
        self.fr_activation = fr_activation
        self.fo_activation = fo_activation
        self.fc_activation = fc_activation
        self.optimizer = optimizer
        self.verbose = verbose
        self.assign_matrices()

        self.Ra = torch.ones(self.Dr, self.Nr)

        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden)
        self.fr2 = nn.Linear(hidden, int(hidden/2))
        self.fr3 = nn.Linear(int(hidden/2), self.De)
        self.fo1 = nn.Linear(self.P + self.Dx + self.De, hidden)
        self.fo2 = nn.Linear(hidden, int(hidden/2))
        self.fo3 = nn.Linear(int(hidden/2), self.Do)
        self.fc1 = nn.Linear(self.Do * self.N, hidden)
        self.fc2 = nn.Linear(hidden, int(hidden/2))
        self.fc3 = nn.Linear(int(hidden/2), self.n_targets)

    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
            self.Rr = Variable(self.Rr)
            self.Rs = Variable(self.Rs)            
            
    def forward(self, x):
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###                                                                                               
        B = torch.transpose(B, 1, 2).contiguous()
        if self.fr_activation ==2:
            B = nn.functional.selu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.selu(self.fr2(B))
            E = nn.functional.selu(self.fr3(B).view(-1, self.Nr, self.De))
        elif self.fr_activation ==1:
            B = nn.functional.elu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.elu(self.fr2(B))
            E = nn.functional.elu(self.fr3(B).view(-1, self.Nr, self.De))
        else:
            B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
            B = nn.functional.relu(self.fr2(B))
            E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###                                                                                              
        if self.fo_activation ==2:
            C = nn.functional.selu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.selu(self.fo2(C))
            O = nn.functional.selu(self.fo3(C).view(-1, self.N, self.Do))
        elif self.fo_activation ==1:
            C = nn.functional.elu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.elu(self.fo2(C))
            O = nn.functional.elu(self.fo3(C).view(-1, self.N, self.Do))
        else:
            C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
            C = nn.functional.relu(self.fo2(C))
            O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        del C
        ### Classification MLP ###                                                                                      
        if self.fc_activation ==2:
            N = nn.functional.selu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.selu(self.fc2(N))
        elif self.fc_activation ==1:
            N = nn.functional.elu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.elu(self.fc2(N))
        else:
            N = nn.functional.relu(self.fc1(O.view(-1, self.Do * self.N)))
            N = nn.functional.relu(self.fc2(N))
        #del O
        #N = nn.functional.relu(self.fc3(N))                                                                            
        N = self.fc3(N)
        return N, O

    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L                                                       
        x_shape = x.size()
        y_shape = y.size()
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

In [17]:
def get_sample(training, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = np.random.choice(ind, 50000)
    return training[chosen_ind], target[chosen_ind]


In [18]:
def accuracy(predict, target):
    _, p_vals = torch.max(predict, 1)
    r = torch.sum(target == p_vals.squeeze(1)).data.numpy()[0]
    t = target.size()[0]
    return r * 1.0 / t

In [19]:
def stats(predict, target):
    print(predict)
    _, p_vals = torch.max(predict, 1)
    t = target.cpu().data.numpy()
    p_vals = p_vals.squeeze(1).data.numpy()
    vals = np.unique(t)
    for i in vals:
        ind = np.where(t == i)
        pv = p_vals[ind]
        correct = sum(pv == t[ind])
        print("  Target %s: %s/%s = %s%%" % (i, correct, len(pv), correct * 100.0/len(pv)))
    print("Overall: %s/%s = %s%%" % (sum(p_vals == t), len(t), sum(p_vals == t) * 100.0/len(t)))
    return sum(p_vals == t) * 100.0/len(t)

In [20]:
nGraphVtx = 30
hidden_nodes = 20
De = 3
Do = 5

In [21]:
#####
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']  # this is a classifier
params = ['j1_px', 'j1_py' , 'j1_pz' , 'j1_e' , 'j1_erel' , 'j1_pt' , 'j1_ptrel', 'j1_eta' , 'j1_etarel' , 
          'j1_etarot' , 'j1_phi' , 'j1_phirel' , 'j1_phirot', 'j1_deltaR' , 'j1_costheta' , 'j1_costhetarel'] # these are the features in the graph

val_split = 0.3
batch_size = 100
n_epochs = 100
patience = 10

In [22]:
nParticles = 30

In [23]:
import glob
#### LIST OF TRAINING FILES
inputTrainFiles = glob.glob("../data/Training/jetImage*_%sp*.h5" %nParticles)
#### LIST OF VALIDATION FILES
inputValFiles = glob.glob("../data/Validation/jetImage*_%sp*.h5" %nParticles)

mymodel = GraphNet(nGraphVtx, len(labels), params, hidden_nodes, De, Do, 0)

In [24]:
loss = nn.CrossEntropyLoss(reduction='mean')
if mymodel.optimizer == 1:        
    optimizer = optim.Adadelta(mymodel.parameters(), lr = 0.0001)
else:
    optimizer = optim.Adam(mymodel.parameters(), lr = 0.0001)

In [25]:
loss_train = np.zeros(n_epochs)
loss_val = np.zeros(n_epochs)
nBatches_per_training_epoch = len(inputTrainFiles)*10000/batch_size
nBatches_per_validation_epoch = len(inputValFiles)*10000/batch_size
print("nBatches_per_training_epoch: %i" %nBatches_per_training_epoch)
print("nBatches_per_validation_epoch: %i" %nBatches_per_validation_epoch)

nBatches_per_training_epoch: 3900
nBatches_per_validation_epoch: 2200


In [26]:
for i in range(n_epochs):
    print(i)
    start = time.time()
    if mymodel.verbose: print("Epoch %s" % i)
    # Define the data generators from the training set and validation set.
    random.shuffle(inputTrainFiles)
    random.shuffle(inputValFiles)
    train_set = InEventLoader(file_names=inputTrainFiles, nP=nParticles,
                              feature_name ='jetConstituentList',label_name = 'jets', verbose=False)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False)
    val_set = InEventLoader(file_names=inputValFiles, nP=nParticles,
                            feature_name ='jetConstituentList',label_name = 'jets', verbose=False)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)
    ####
    # train
    for batch_idx, mydict in enumerate(train_loader):
        data = mydict['jetConstituentList']
        target = mydict['jets']
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        out, hidden = mymodel(data)
        l = loss(out, target)
        l.backward()
        optimizer.step()
        loss_train[i] += l.cpu().data.numpy()/nBatches_per_training_epoch
    # validation
    for batch_idx, mydict in enumerate(val_loader):
        data = mydict['jetConstituentList']
        target = mydict['jets']
        data, target = Variable(data, volatile=True), Variable(target)
        out_val, hidden = mymodel(data)
        l_val = loss(out_val, target)
        loss_val[i] += l_val.cpu().data.numpy()/nBatches_per_validation_epoch
    if mymodel.verbose: print("Training   Loss: %f" %loss_train[i])
    if mymodel.verbose: print("Validation Loss: %f" %loss_val[i])
    if all(loss_val[max(0, i - patience):i] > min(np.append(loss_val[0:max(0, i - patience)], 200))) and i > patience:
        print("Early Stopping")
        break
    stop = time.time()
    duration = start - stop
    print(duration)
    

0


C:\Users\anrun\anaconda3\envs\EPE_ML\lib\site-packages\ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


-303.71861457824707
1
-300.45164155960083
2
-302.4614453315735
3
-305.07146883010864
4
-308.77799701690674
5
-301.9152693748474
6
-315.9819757938385
7
-339.23446249961853
8
-329.7103018760681
9
-325.25540113449097
10
-364.78473114967346
11
-349.32059693336487
12
-365.6239881515503
13
-322.428395986557
14
-326.0940935611725
15
-325.25250482559204
16
-322.30002331733704
17
-312.29039454460144
18
-308.01187658309937
19
-310.6941719055176
20
-296.66531562805176
21
-297.48386096954346
22
-299.8619635105133
23
-295.04224944114685
24
-297.6647391319275
25
-295.13219571113586
26
-295.0752286911011
27
-299.490642786026
28
-296.35049533843994
29
-294.8173773288727
30
-295.3080973625183
31
-295.2561230659485
32
-294.9713077545166
33
-294.90730690956116
34
-294.7803997993469
35
-294.3906216621399
36
-295.3150911331177
37
-294.04482197761536
38
-293.23928594589233
39
-293.32023882865906
40
-292.9984242916107
41
-297.15902972221375
42
-295.2591230869293
43
-294.94030714035034
44
-295.48599338531494


In [81]:
import h5py
f = h5py.File("firsttryhistory.h5", "w")
f.create_dataset('train_loss', data= np.asarray(loss_train), compression='gzip')
f.create_dataset('val_loss', data= np.asarray(loss_val), compression='gzip')

# the best model
torch.save(mymodel.state_dict(), 'C:/Users/anrun/JEDInet-code/models/best_model.params')


In [58]:
finalloss=[]
models=[]
models.append(mymodel)
loss_val = loss_val[loss_val>0]
finalloss.append(loss_val[-1])

[0.7125085610151303]

In [85]:
# Load
X_test = np.array([])
Y_test = np.array([])
inputFiles = glob.glob("../data/Validation/jetImage*_%sp*.h5" %nParticles)   
#inputFiles = glob.glob("/data/ML/mpierini/hls-fml/VALIDATION/jetImage_9_%sp*.h5" %nParticles)
#inputFiles = glob.glob("/data/ml/mpierini/hls-fml/VALIDATION/jetImage_9_%sp*.h5" %nParticles)
random.shuffle(inputFiles)
for fileINname in inputFiles:
    print(fileINname)
    f = h5py.File(fileINname, 'r')
    myFeatures = np.array(f.get('jetConstituentList'))
    myTarget = np.array(f.get('jets')[0:,-6:-1])
    print(myFeatures.size)
    X_test = np.concatenate([X_test,myFeatures], axis = 0) if X_test.size else myFeatures
    Y_test = np.concatenate([Y_test,myTarget], axis = 0) if Y_test.size else myTarget
    print(X_test.shape, Y_test.shape)


../data/Validation\jetImage_5_30p_20000_30000.h5
4800000
(10000, 30, 16) (10000, 5)
../data/Validation\jetImage_5_30p_40000_50000.h5
4800000
(20000, 30, 16) (20000, 5)
../data/Validation\jetImage_4_30p_50000_60000.h5
4800000
(30000, 30, 16) (30000, 5)
../data/Validation\jetImage_6_30p_10000_20000.h5
4800000
(40000, 30, 16) (40000, 5)
../data/Validation\jetImage_5_30p_80000_90000.h5
4800000
(50000, 30, 16) (50000, 5)
../data/Validation\jetImage_6_30p_50000_60000.h5
4800000
(60000, 30, 16) (60000, 5)
../data/Validation\jetImage_6_30p_70000_80000.h5
4800000
(70000, 30, 16) (70000, 5)
../data/Validation\jetImage_6_30p_40000_50000.h5
4800000
(80000, 30, 16) (80000, 5)
../data/Validation\jetImage_6_30p_80000_90000.h5
4800000
(90000, 30, 16) (90000, 5)
../data/Validation\jetImage_6_30p_30000_40000.h5
4800000
(100000, 30, 16) (100000, 5)
../data/Validation\jetImage_5_30p_30000_40000.h5
4800000
(110000, 30, 16) (110000, 5)
../data/Validation\jetImage_6_30p_60000_70000.h5
4800000
(120000, 30, 16

In [49]:
inputFiles

['../data/Training\\jetImage_4_30p_20000_30000.h5',
 '../data/Training\\jetImage_3_30p_80000_90000.h5',
 '../data/Training\\jetImage_0_30p_10000_20000.h5',
 '../data/Training\\jetImage_1_30p_50000_60000.h5',
 '../data/Training\\jetImage_2_30p_0_10000.h5',
 '../data/Training\\jetImage_1_30p_20000_30000.h5',
 '../data/Training\\jetImage_3_30p_30000_40000.h5',
 '../data/Training\\jetImage_1_30p_70000_80000.h5',
 '../data/Training\\jetImage_3_30p_0_10000.h5',
 '../data/Training\\jetImage_1_30p_60000_70000.h5',
 '../data/Training\\jetImage_3_30p_10000_20000.h5',
 '../data/Training\\jetImage_2_30p_70000_80000.h5',
 '../data/Training\\jetImage_0_30p_80000_90000.h5',
 '../data/Training\\jetImage_2_30p_20000_30000.h5',
 '../data/Training\\jetImage_3_30p_20000_30000.h5',
 '../data/Training\\jetImage_4_30p_10000_20000.h5',
 '../data/Training\\jetImage_3_30p_60000_70000.h5',
 '../data/Training\\jetImage_4_30p_30000_40000.h5',
 '../data/Training\\jetImage_0_30p_60000_70000.h5',
 '../data/Training\\

In [86]:
# shuffle and split
X_test, Y_test = shuffle(X_test, Y_test, random_state=1)

X_test = np.swapaxes(X_test, 1, 2)
Y_test = np.argmax(Y_test, axis=1)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)


In [99]:
# extract the O matrix and the category output [TBF]
predict_test = []
lst = []
Otot = []
for j in torch.split(X_test, batch_size):
    a, myO = mymodel(j)
    a = a.cpu().data.numpy()
    myO = torch.sum(myO, dim=1)
    myO = myO.cpu().data.numpy()
    # sum over particles
    lst.append(a)
    Otot.append(myO)
    
predicted = Variable(torch.FloatTensor(np.concatenate(lst)))
predicted = torch.nn.functional.softmax(predicted, dim=1)
predict_test = predicted.data.numpy()

O_predicted = Variable(torch.FloatTensor(np.concatenate(Otot)))
O_predicted_test = O_predicted.data.numpy()

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [76]:
np.shape(lst)

(3900,)

In [94]:
#### get the ROC curves


AssertionError: Torch not compiled with CUDA enabled